In [1]:
import numpy as np
import pandas as pd

In [2]:
Z = 26
A = 56
mass_nucleon = 0.938273
mass_nucleus = A*0.931494
alpha_fine = 1/137
system_err = 0

qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']
dataSet_to_name = {"Hotta:1994":"Hotta:1994","Hotta:1984":"Hotta:1984"}
dataSet_to_normalization = {"Hotta:1994":1,"Hotta:1984":1}
dataSet_to_normError = {"Hotta:1994":0,"Hotta:1984":0}

def prepare_df(filepath):

    df = pd.read_csv(filepath)

    df["normalization"]=df["dataSet"].map(dataSet_to_normalization)
    df["normError"]=df["dataSet"].map(dataSet_to_normError)
    df['normCross'] = df['cross']*df['normalization']
    df['system_err'] = 0.05
    df['error'] = np.sqrt(df['error']**2 + ((df['system_err']*df['cross'])**2))
    df['normCrossError']=df['normCross']*np.sqrt((df['error']/df['cross'])**2+(df['normError']/df['normalization'])**2)
    
    df["ThetaRad"]=df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"]=(np.sin(df["ThetaRad"]/2))**2
    df["cos2(T/2)"]=(np.cos(df["ThetaRad"]/2))**2
    df["tan2(T/2)"]=(np.tan(df["ThetaRad"]/2))**2
    df["nuel"]=df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_nucleus)
    df["Ex"]=df["nu"]-df["nuel"]
    df["Ep"]=df["E0"]-df["nu"]

    df["R"]=1.1*A**(1/3)+0.86/(A**(1/3))
    df["Veff"]=0.0089
    
    df["Eeff"]=df["E0"]+df["Veff"]
    df["Ep_eff"]=df["Ep"]+df["Veff"]

    df["F2foc"]=(df["Eeff"]/df["E0"])**2
    df["Q2"]=4*df["E0"]*(df["Ep"])*df["sin2(T/2)"]
    df["Q2eff"]=4*df["Eeff"]*df["Ep_eff"]*df["sin2(T/2)"]
    df["qv"]=np.sqrt(df["nu"]**2+df["Q2eff"])
    df["W2"]=mass_nucleon**2+2*mass_nucleon*df["nu"]-df["Q2eff"]

    df['RT']=(1/df["F2foc"])*df["normCross"]*((df["Q2eff"]**2)/(4*(alpha_fine**2)*(df["Ep_eff"]**2)))/((0.1973269**2)*10000000)
    df['RTerr'] = df['RT'] * (df['normCrossError'] / df['normCross'])

    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"]=pd.cut(x=df["qv"],bins=qvbins,labels=qvbin_names,right=True)
    df["qvcenter"]=pd.cut(x=df["qv"],bins=qvbins,labels=qvcenters,right=True)
    df['qvcenter']=pd.to_numeric(df['qvcenter'])
    df["Q2bin"]=pd.cut(x=df["Q2eff"],bins=Q2bins,labels=Q2bin_names,right=True)
    df["Q2center"]=pd.cut(x=df["Q2eff"],bins=Q2bins,labels=Q2centers,right=True)
    df['Q2center']=pd.to_numeric(df['Q2center'])
    df=df.dropna()

    return df

df = prepare_df('Fe56_Hotta.csv')
df

,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,normalization,normError,...,Q2,Q2eff,qv,W2,RT,RTerr,qvbin,qvcenter,Q2bin,Q2center
0,26,56,0.1546,180,0.0058,459.0,27.417194,Hotta:1994,1,0,...,0.092018,0.103136,0.3212,0.788104,2.115243,0.126349,"[0.270,0.340]",0.300,"[0.100,0.145]",0.120
1,26,56,0.1546,180,0.0158,4120.0,212.960090,Hotta:1994,1,0,...,0.085834,0.096596,0.3112,0.813410,18.986492,0.981399,"[0.270,0.340]",0.300,"[0.070,0.100]",0.093
2,26,56,0.1546,180,0.0258,7080.0,363.148730,Hotta:1994,1,0,...,0.079650,0.090056,0.3012,0.838715,32.627273,1.673524,"[0.270,0.340]",0.300,"[0.070,0.100]",0.093
3,26,56,0.1546,180,0.0357,10000.0,508.759275,Hotta:1994,1,0,...,0.073528,0.083581,0.2913,0.863768,46.083719,2.344552,"[0.270,0.340]",0.300,"[0.070,0.100]",0.093
4,26,56,0.1546,180,0.0457,12500.0,635.856902,Hotta:1994,1,0,...,0.067344,0.077041,0.2813,0.889073,57.604649,2.930265,"[0.270,0.340]",0.300,"[0.070,0.100]",0.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,26,56,0.5553,180,0.4006,1410.0,109.358173,Hotta:1984,1,0,...,0.343620,0.369212,0.7278,1.262888,83.830682,6.501823,"[0.702,0.878]",0.756,"[0.322,0.438]",0.380
217,26,56,0.5553,180,0.4154,1800.0,139.817739,Hotta:1984,1,0,...,0.310746,0.335812,0.7130,1.324062,107.017892,8.312778,"[0.702,0.878]",0.756,"[0.322,0.438]",0.380
218,26,56,0.5553,180,0.4200,1860.0,140.264037,Hotta:1984,1,0,...,0.300528,0.325431,0.7084,1.343075,110.585155,8.339312,"[0.702,0.878]",0.756,"[0.322,0.438]",0.380
219,26,56,0.5553,180,0.4254,2070.0,159.227039,Hotta:1984,1,0,...,0.288534,0.313244,0.7030,1.365395,123.070576,9.466746,"[0.702,0.878]",0.756,"[0.206,0.322]",0.265


In [3]:
zero_rows = df.groupby(["E0", "ThetaDeg"]).first().reset_index()[["E0", "ThetaDeg"]]
zero_rows["nu"] = 0
zero_rows["RT"] = 0
zero_rows["RTerr"] = 0.1

df = pd.concat([df, zero_rows], ignore_index=True)

In [4]:
df.to_csv('df_Fe56_Hotta_raw.csv', index=False)

In [5]:
def fit_y(x, y, degree=8, x_pred=None):
    coeffs = np.polyfit(x, y, degree)
    poly_func = np.poly1d(coeffs)
    y_pred = poly_func(x_pred) if x_pred is not None else None    
    return y_pred

def fit_yerr(df, target):
    closest_idx = (df['nu'] - target).abs().idxmin()
    yerr = df.loc[closest_idx, 'RTerr'] / 2
    return yerr

df_fit = pd.read_csv("fit_Fe56_Hotta.csv")
df_fit["ThetaRad"] = df_fit["ThetaDeg"] * np.pi/180
df_fit["sin2(T/2)"] = (np.sin(df_fit["ThetaRad"] / 2))**2
df_fit["Veff"] = 0.0089   
df_fit["Eeff"] = df_fit["E0"] + df_fit["Veff"]
df_fit.loc[df_fit["qvcenter"].notna(), "nu"] = 2 * df_fit["Eeff"] * df_fit['sin2(T/2)'] - np.sqrt((4 * df_fit["Eeff"]**2) * (df_fit["sin2(T/2)"]**2 - df_fit["sin2(T/2)"]) + df_fit["qvcenter"]**2)
df_fit.loc[df_fit["Q2center"].notna(), "nu"] = df_fit["Eeff"] - (df_fit['Q2center'] / (4 * df_fit["Eeff"] * df_fit["sin2(T/2)"] ))

for (E0, ThetaDeg) in df_fit[df_fit["qvcenter"].notna()][["E0", "ThetaDeg"]].drop_duplicates().itertuples(index=False):
    df_filter = df[(df["E0"] == E0) & (df["ThetaDeg"] == ThetaDeg)]
    df_fit_filter = df_fit[(df_fit["E0"] == E0) & (df_fit["ThetaDeg"] == ThetaDeg)].copy()
    if len(df_filter) > 2 and not df_fit_filter.empty:
        df_fit.loc[df_fit_filter.index, "RT"] = fit_y(df_filter["nu"], df_filter["RT"], x_pred=df_fit_filter["nu"])
        df_fit.loc[df_fit_filter.index, "RTerr"] = df_fit_filter["nu"].apply(lambda nu: fit_yerr(df_filter, nu))
     
for (E0, ThetaDeg) in df_fit[df_fit["Q2center"].notna()][["E0", "ThetaDeg"]].drop_duplicates().itertuples(index=False):
    df_filter = df[(df["E0"] == E0) & (df["ThetaDeg"] == ThetaDeg)]
    df_fit_filter = df_fit[(df_fit["E0"] == E0) & (df_fit["ThetaDeg"] == ThetaDeg)].copy()
    if len(df_filter) > 2 and not df_fit_filter.empty:
        df_fit.loc[df_fit_filter.index, "RT"] = fit_y(df_filter["nu"], df_filter["RT"], x_pred=df_fit_filter["nu"])
        df_fit.loc[df_fit_filter.index, "RTerr"] = df_fit_filter["nu"].apply(lambda nu: fit_yerr(df_filter, nu))

# df['nu'] = df["Eeff"] - (df['q2center'] / (4 * df["Eeff"] * df["sin2(T/2)"] ))
# df['nu'] = 2 * df["Eeff"] * df['sin2(T/2)'] + np.sqrt(4 * (df["Eeff"]**2) * (df["sin2(T/2)"]**2 - df["sin2(T/2)"]) + df["qvcenter"]**2)


In [6]:
df_fit.to_csv('df_Fe56_Hotta.csv', index=False)